In [6]:
from nilearn.reporting import get_clusters_table
from nilearn import input_data
from nilearn import image


In [7]:
filepath = '/home/hyruuk/GitHub/neuromod/shinobi_fmri/data/processed/cmaps/run-level/Hit/sub-01_ses-002_run-01.nii.gz'

In [8]:
import nibabel as nib
img = nib.load(filepath)
arrdata = img.get_fdata()

In [9]:
import os
import pandas as pd
from nilearn import image, signal
from load_confounds import Confounds
from shinobi_fmri.annotations.annotations import trim_events_df, get_scrub_regressor
import numpy as np
import pdb
import argparse
import nilearn
import shinobi_behav
from nilearn.glm.first_level import make_first_level_design_matrix, FirstLevelModel
from nilearn.glm import threshold_stats_img
from nilearn import plotting

figures_path = shinobi_behav.figures_path#'/home/hyruuk/GitHub/neuromod/shinobi_fmri/reports/figures/'
path_to_data = shinobi_behav.path_to_data#'/media/storage/neuromod/shinobi_data/'
sub = 'sub-01'
contrast = 'Hit'
ses = 'ses-002'
run = '1'

print('Run : {}'.format(run))
data_fname = path_to_data + 'shinobi/derivatives/fmriprep-20.2lts/fmriprep/{}/{}/func/{}_{}_task-shinobi_run-{}_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz'.format(sub, ses, sub, ses, run)
confounds_fname = path_to_data + 'shinobi/derivatives/fmriprep-20.2lts/fmriprep/{}/{}/func/{}_{}_task-shinobi_run-{}_desc-confounds_timeseries.tsv'.format(sub, ses, sub, ses, run)
anat_fname = path_to_data + 'anat/derivatives/fmriprep-20.2lts/fmriprep/{}/anat/{}_space-MNI152NLin2009cAsym_desc-brain_mask.nii.gz'.format(sub, sub)
cmap_fname = path_to_data + 'processed/cmaps/run-level-allregs/{}/{}_{}_run-0{}.nii.gz'.format(contrast, sub, ses, run)
events_fname = path_to_data + 'processed/annotations/{}_{}_run-0{}.csv'.format(sub, ses, run)

run_events = pd.read_csv(events_fname)

fmri_img = image.concat_imgs(data_fname)
bold_shape = fmri_img.shape
confounds = Confounds(strategy=['high_pass', 'motion', 'global', 'wm_csf'],
                            motion="full", wm_csf='basic',
                            global_signal='full').load(data_fname)


hrf_model = 'spm'
trimmed_df = trim_events_df(run_events, trim_by='event')

# create design matrix
n_slices = bold_shape[-1]
t_r = 1.49
frame_times = np.arange(n_slices) * t_r
design_matrix = make_first_level_design_matrix(frame_times,
                                        events=trimmed_df,
                                        drift_model=None,
                                        hrf_model=hrf_model,
                                        add_regs=confounds,
                                        add_reg_names=None)
design_matrix = get_scrub_regressor(run_events, design_matrix)
fmri_glm = FirstLevelModel(t_r=1.49,
                       noise_model='ar1',
                       standardize=False,
                       hrf_model=hrf_model,
                       drift_model=None,
                       high_pass=None,
                       n_jobs=16,
                       smoothing_fwhm=5,
                       mask_img=anat_fname,
                       minimize_memory=False)

fmri_glm = fmri_glm.fit(fmri_img, design_matrices=design_matrix)
z_map = fmri_glm.compute_contrast(contrast,
                          stat_type='F',
                          output_type='z_score')

# Extract activation clusters
from nilearn.reporting import get_clusters_table
from nilearn import input_data

table = get_clusters_table(z_map, stat_threshold=3.1,
                           cluster_threshold=20).set_index('Cluster ID', drop=True)

# get the 3 largest clusters' max x, y, and z coordinates
coords = table.loc[range(1, 4), ['X', 'Y', 'Z']].values

# extract time series from each coordinate
masker = input_data.NiftiSpheresMasker(coords)
real_timeseries = masker.fit_transform(fmri_img)
predicted_timeseries = masker.fit_transform(fmri_glm.predicted[0])

ModuleNotFoundError: No module named 'shinobi_behav'

In [14]:
pwd

'/home/hyruuk/GitHub/neuromod/shinobi_fmri/notebooks'

In [ ]:
import shinobi_behav# Importe les librairies
from nilearn.datasets import fetch_spm_auditory
from nilearn import image
from nilearn import masking
import pandas as pd

# initialisation de la figure
fig = plt.figure(figsize=(7,5))


# Extract activation clusters
from nilearn.reporting import get_clusters_table
from nilearn import input_data

table = get_clusters_table(z_map, stat_threshold=3.1,
                           cluster_threshold=20).set_index('Cluster ID', drop=True)

# get the 3 largest clusters' max x, y, and z coordinates
coords = table.loc[range(1, 4), ['X', 'Y', 'Z']].values

# extract time series from each coordinate
masker = input_data.NiftiSpheresMasker(coords)
real_timeseries = masker.fit_transform(fmri_img)
predicted_timeseries = masker.fit_transform(fmri_glm.predicted[0])

# Plot figure
# colors for each of the clusters
colors = ['blue', 'navy', 'purple', 'magenta', 'olive', 'teal']
# plot the time series and corresponding locations
from nilearn import plotting
fig1, axs1 = plt.subplots(2, 3)
for i in range(0, 3):
    # plotting time series
    axs1[0, i].set_title('Cluster peak {}\n'.format(coords[i]))
    axs1[0, i].plot(real_timeseries[:, i], c=colors[i], lw=2)
    axs1[0, i].plot(predicted_timeseries[:, i], c='r', ls='--', lw=2)
    axs1[0, i].set_xlabel('Time')
    axs1[0, i].set_ylabel('Signal intensity', labelpad=0)
    # plotting image below the time series
    roi_img = plotting.plot_stat_map(
        z_map, cut_coords=[coords[i][2]], threshold=3.1, figure=fig1,
        axes=axs1[1, i], display_mode='z', colorbar=False, bg_img=mean_img)
    roi_img.add_markers([coords[i]], colors[i], 300)

fig1.set_size_inches(24, 14)
signals_plot_name = figures_path + 'run-level-allregs/{}/signals_{}_{}_run-0{}.png'.format(contrast, sub, ses, run)
fig1.savefig(signals_plot_name)
# Glue the figure
#from myst_nb import glue
#glue("auditory-fig", fig1, display=False)